<a href="https://colab.research.google.com/github/AchmadRendiSetiawan/2300441100018/blob/main/Data_Stunting_STATISTIKA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load Library & Dataset Awal

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, matthews_corrcoef
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [4]:
# Load dataset
file_path = '/content/drive/MyDrive/SEMESTER 5/UAS - STATISTIKA/Data Stunting - Statistika.xlsx'

try:
    df = pd.read_excel(file_path)
except FileNotFoundError:
    print("File not found. Mounting Google Drive...")
    from google.colab import drive
    drive.mount('/content/drive')
    # Try loading the file again after mounting
    try:
        df = pd.read_excel(file_path)
        print("File loaded successfully after mounting Google Drive.")
    except FileNotFoundError:
        print(f"Error: File still not found at {file_path} after mounting Google Drive. Please check the file path.")
except Exception as e:
    print(f"An error occurred: {e}")

File not found. Mounting Google Drive...
Mounted at /content/drive
File loaded successfully after mounting Google Drive.


Data Understanding

In [5]:
# Menampilkan 5 data pertama
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Data Posyandu Pengaruh Stunting di desa Gempol...,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,No,Bulan,Nama Orang Tua,Nama Anak,Usia,Tinggi Badan,Berat Badan
4,NaN,1,Januari,Yuni/Suwadi,M. Ardian,9,97,12.1


In [ ]:
# Informasi pada data
df.info()

In [ ]:
df.describe()

Data Visualization (Eksplorasi Awal / Distribusi Data)

In [ ]:
# Distribusi Tinggi Badan dan Berat Badan
fig, axes = plt.subplots(2, 1, figsize=(8, 8))
fig.suptitle("Distribusi Tinggi Badan dan Berat Badan Anak", fontsize=16, fontweight="bold")

sns.histplot(df["Tinggi Badan"], color="skyblue", ax=axes[0], bins=8, edgecolor="black")
axes[0].set_title("Distribusi Tinggi Badan", fontsize=14)
axes[0].set_xlabel("Tinggi Badan (cm)")
axes[0].set_ylabel("Jumlah Anak")

sns.histplot(df["Berat Badan"], color="salmon", ax=axes[1], bins=8, edgecolor="black")
axes[1].set_title("Distribusi Berat Badan", fontsize=14)
axes[1].set_xlabel("Berat Badan (kg)")
axes[1].set_ylabel("Jumlah Anak")

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

In [ ]:
# Distribusi Class
target_col = "Class"
if target_col in df.columns:
    plt.figure(figsize=(6,4))
    sns.countplot(x=target_col, data=df, palette='Set2')
    plt.title(f"Distribusi {target_col}")
    plt.show()

In [ ]:
df.columns = df.columns.str.strip()

In [ ]:
# Distribusi jumlah anak berdasarkan usia
plt.figure(figsize=(8,5))
order_usia = sorted(df["Usia"].dropna().unique())
sns.countplot(x="Usia", data=df, order=order_usia, palette="viridis")

plt.title("Distribusi Jumlah Anak Berdasarkan Usia", fontsize=14)
plt.xlabel("Usia (Tahun)", fontsize=12)
plt.ylabel("Jumlah Anak", fontsize=12)
plt.show()

Data Preprocessing

In [ ]:
print("Total baris awal (df):", len(df))
print("Kolom tersedia:", df.columns.tolist())

cols = ['Usia','Tinggi Badan','Berat Badan','Class']
df_sel = df.copy()

# Cek kolom
for c in cols:
    print(c, "in df_sel?", c in df_sel.columns)

df_sel = df_sel[cols]
print("\nMissing per kolom:\n", df_sel.isnull().sum())

# Cek baris yang missing value
print("\nContoh baris dengan missing:")
print(df_sel[df_sel.isnull().any(axis=1)].head(10))

# Membersihkan spasi
if df_sel['Class'].dtype == object:
    print("\nUnique label sebelum strip:", df_sel['Class'].unique()[:20])
    df_sel['Class'] = df_sel['Class'].astype(str).str.strip()
    print("Unique label setelah strip:", df_sel['Class'].unique()[:20])

df_sel = df_sel.reset_index(drop=True)
print("\nTotal baris setelah select & reset:", len(df_sel))

# Jika ada missing, nilai jumlah yang terbuang
n_before = len(df_sel)
df_dropna = df_sel.dropna().reset_index(drop=True)
n_after = len(df_dropna)
print("Baris hilang kalau dropna():", n_before - n_after)
print("Total setelah dropna():", n_after)

# Cek jika ada nilai non-numerik di fitur
for f in ['Usia','Tinggi Badan','Berat Badan']:
    non_num = df_dropna[~df_dropna[f].apply(lambda x: isinstance(x, (int,float,np.integer,np.floating)))]
    print(f, "jumlah non-numeric contoh:", len(non_num))
    if len(non_num)>0:
        print(non_num.head())

Arsitektur Model

In [ ]:
df = df[['Usia', 'Tinggi Badan', 'Berat Badan', 'Class']]

X = df[['Usia', 'Tinggi Badan', 'Berat Badan']]
y = df['Class']

# Encode target
if y.dtype == 'object':
    le = LabelEncoder()
    y = le.fit_transform(y)

# Standarisasi data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Definisikan model
models = {
    "kNN": KNeighborsClassifier(n_neighbors=3),
    "Naive Bayes": GaussianNB(),
    "Random Forest": RandomForestClassifier(random_state=42)
}

# Cross Validation (5-Fold Stratified)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Evaluasi setiap model
results = []

for name, model in models.items():
    aucs, accs, f1s, precs, recalls, mccs = [], [], [], [], [], []

    for train_index, test_index in kf.split(X_scaled, y):
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else model.decision_function(X_test)

        aucs.append(roc_auc_score(y_test, y_prob))
        accs.append(accuracy_score(y_test, y_pred))
        f1s.append(f1_score(y_test, y_pred))
        precs.append(precision_score(y_test, y_pred))
        recalls.append(recall_score(y_test, y_pred))
        mccs.append(matthews_corrcoef(y_test, y_pred))

    results.append({
        "Model": name,
        "AUC": round(np.mean(aucs), 3),
        "CA": round(np.mean(accs), 3),
        "F1": round(np.mean(f1s), 3),
        "Prec": round(np.mean(precs), 3),
        "Recall": round(np.mean(recalls), 3),
        "MCC": round(np.mean(mccs), 3)
    })

# Tampilkan hasil evaluasi (rata-rata 5 fold)
results_df = pd.DataFrame(results)
print("=== Hasil Evaluasi Model ===\n")
print(results_df)

In [ ]:
# Confusion Matrix untuk setiap model
for name, model in models.items():
    y_true_all, y_pred_all = [], []

    for train_index, test_index in kf.split(X_scaled, y):
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        y_true_all.extend(y_test)
        y_pred_all.extend(y_pred)

    # Buat confusion matrix
    cm = confusion_matrix(y_true_all, y_pred_all)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
    disp.plot(cmap='Blues', colorbar=True)
    plt.title(f'Confusion Matrix - {name}')
    plt.show()

    print("\n")

In [ ]:
# ROC Analysis
plt.figure(figsize=(8,6))

for name, model in models.items():
    y_true_all, y_prob_all = [], []

    for train_index, test_index in kf.split(X_scaled, y):
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.fit(X_train, y_train)

        if hasattr(model, "predict_proba"):
            y_prob = model.predict_proba(X_test)[:, 1]
        else:
            y_prob = model.decision_function(X_test)

        y_true_all.extend(y_test)
        y_prob_all.extend(y_prob)

    fpr, tpr, _ = roc_curve(y_true_all, y_prob_all)
    roc_auc = auc(fpr, tpr)

    plt.plot(fpr, tpr, lw=2, label=f"{name} (AUC = {roc_auc:.3f})")

plt.plot([0, 1], [0, 1], color='gray', lw=1.5, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Analysis')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()